In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import warnings
import csv
import yaml
import json
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Any, TypedDict

from megadetector.detection.run_detector import load_detector, model_string_to_model_version
from megadetector.detection.run_detector_batch import process_images, write_results_to_file

from os import PathLike
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

from ba_dev.dataloader import MammaliaData
from ba_dev.utils import load_config_yaml

paths = load_config_yaml('../path_config.yml')


### Running Tests

In [4]:
path_to_dataset = paths['dataset']
path_labelfiles = paths['testset']
path_to_detector_output = path_labelfiles / 'md_out'
detector_model = 'mdv5a'
mode = 'init'

dataset = MammaliaData(
    path_to_dataset = path_to_dataset,
    path_labelfiles = path_labelfiles,
    path_to_detector_output = path_to_detector_output,
    detector_model = detector_model,
    mode = mode,
    )

Bypassing download of already-downloaded file md_v5a.0.0.pt
Model v5a.0.0 available at /var/folders/t0/5px6w93n5rvdnqbxvt4n6qf00000gn/T/megadetector_models/md_v5a.0.0.pt
Loading PT detector with compatibility mode classic


Fusing layers... 
Fusing layers... 
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
/Users/jk/MyPythonPackages/BA_package/ba_dev/dataloader.py:219: UserWarning: With the detection confidence of 0.25,
8 sequences of the full set had no detections and will be excluded.
Excluded sequences: [6000709, 2000045, 6000974, 6000145, 6000019, 6000276, 6000924, 6000318]
  warnings.warn(


### Sampling dataset

In [3]:
# Paths
path_to_labelfiles = paths['labels']
dataset_root = paths['dataset']
target_dir = paths['testset']
output_metadata_csv = target_dir / "metadata_sample_set.csv"


files = path_to_labelfiles.glob('*.csv')
metadata = pd.DataFrame()
categories_to_drop = ['other', 'glis_glis']

for file in files:
    metadata = pd.concat([metadata, pd.read_csv(file)], ignore_index=True)
    metadata = metadata.dropna(subset=['label2'])
    metadata = metadata[~metadata['label2'].isin(categories_to_drop)]


metadata_filtered = metadata[metadata['n_files']<60]

metadata_sampled = metadata_filtered.groupby("label2", group_keys=False).sample(n=40, random_state=42)

metadata_sampled.to_csv(output_metadata_csv, index=False)


In [20]:
paths['labels']

PosixPath('/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels')